# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [67]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]
n_features

296

As a refresher, preview the training data: 

In [68]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [5]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units

baseline_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
# Hidden layer with 50 units

baseline_model.add(layers.Dense(50, activation='relu'))


# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 144us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
1051/1051 [==============================] - 0s 52us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
1051/1051 [==============================] - 0s 57us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
1051/1051 [==============================] - 0s 56us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
1051/1051 [==============================] - 0s 66us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
1051/1051 [==============================] - 0s 59us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
1051/1051 [==============================] - 0s 51us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
1051/1051 [==============================] - 0s 67us/step - loss: nan - 

1051/1051 [==============================] - 0s 53us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 67/150
1051/1051 [==============================] - 0s 48us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 68/150
1051/1051 [==============================] - 0s 51us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 69/150
1051/1051 [==============================] - 0s 54us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
1051/1051 [==============================] - 0s 48us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
1051/1051 [==============================] - 0s 49us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
1051/1051 [==============================] - 0s 46us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
1051/1051 [==============================] - 0s 48us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150


1051/1051 [==============================] - 0s 51us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 132/150
1051/1051 [==============================] - 0s 55us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 133/150
1051/1051 [==============================] - 0s 51us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 134/150
1051/1051 [==============================] - 0s 52us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 135/150
1051/1051 [==============================] - 0s 47us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 136/150
1051/1051 [==============================] - 0s 46us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
1051/1051 [==============================] - 0s 47us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
1051/1051 [==============================] - 0s 46us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [7]:
# Numeric column names
numeric_columns = X_train_numeric.columns

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [6]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [8]:
# Train the model 
normalized_input_model.fit(X_train, y_train,
                          batch_size=32,
                          epochs=150, 
                          validation_data=(X_val, y_val))


Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 102us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
1051/1051 [==============================] - 0s 51us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
1051/1051 [==============================] - 0s 56us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
1051/1051 [==============================] - 0s 64us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
1051/1051 [==============================] - 0s 60us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
1051/1051 [==============================] - 0s 56us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
1051/1051 [==============================] - 0s 70us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
1051/1051 [==============================] - 0s 57us/step - loss: nan - 

1051/1051 [==============================] - 0s 45us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 67/150
1051/1051 [==============================] - 0s 45us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 68/150
1051/1051 [==============================] - 0s 44us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 69/150
1051/1051 [==============================] - 0s 44us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
1051/1051 [==============================] - 0s 47us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
1051/1051 [==============================] - 0s 44us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
1051/1051 [==============================] - 0s 45us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
1051/1051 [==============================] - 0s 42us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150


1051/1051 [==============================] - 0s 66us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 132/150
1051/1051 [==============================] - 0s 66us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 133/150
1051/1051 [==============================] - 0s 62us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 134/150
1051/1051 [==============================] - 0s 54us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 135/150
1051/1051 [==============================] - 0s 49us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 136/150
1051/1051 [==============================] - 0s 61us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 137/150
1051/1051 [==============================] - 0s 55us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
1051/1051 [==============================] - 0s 50us/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [70]:
y_train.shape

(1051, 1)

In [71]:
y_test.shape

(146, 1)

In [9]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [10]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
# normalized_model.fit(X_train, 
#                      y_train_scaled,
#                      batch_size=32,
#                      epochs=150, 
#                      validation_data=(X_val, y_val_scaled))
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))


Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 102us/step - loss: 0.5035 - mse: 0.5035 - val_loss: 0.2219 - val_mse: 0.2219
Epoch 2/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.2178 - mse: 0.2178 - val_loss: 0.1752 - val_mse: 0.1752
Epoch 3/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.1802 - mse: 0.1802 - val_loss: 0.1624 - val_mse: 0.1624
Epoch 4/150
1051/1051 [==============================] - 0s 54us/step - loss: 0.1618 - mse: 0.1618 - val_loss: 0.1394 - val_mse: 0.1394
Epoch 5/150
1051/1051 [==============================] - 0s 58us/step - loss: 0.1465 - mse: 0.1465 - val_loss: 0.1311 - val_mse: 0.1311
Epoch 6/150
1051/1051 [==============================] - 0s 59us/step - loss: 0.1325 - mse: 0.1325 - val_loss: 0.1258 - val_mse: 0.1258
Epoch 7/150
1051/1051 [==============================] - 0s 56us/step - loss: 0.1259 - mse: 0.1259 - val_loss: 0.1265 - val_mse: 0.1265


1051/1051 [==============================] - 0s 63us/step - loss: 0.0273 - mse: 0.0273 - val_loss: 0.1364 - val_mse: 0.1364
Epoch 61/150
1051/1051 [==============================] - 0s 61us/step - loss: 0.0276 - mse: 0.0276 - val_loss: 0.1344 - val_mse: 0.1344
Epoch 62/150
1051/1051 [==============================] - 0s 62us/step - loss: 0.0265 - mse: 0.0265 - val_loss: 0.1403 - val_mse: 0.1403
Epoch 63/150
1051/1051 [==============================] - 0s 63us/step - loss: 0.0271 - mse: 0.0271 - val_loss: 0.1420 - val_mse: 0.1420
Epoch 64/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.0268 - mse: 0.0268 - val_loss: 0.1387 - val_mse: 0.1387
Epoch 65/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0257 - mse: 0.0257 - val_loss: 0.1379 - val_mse: 0.1379
Epoch 66/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0249 - mse: 0.0249 - val_loss: 0.1397 - val_mse: 0.1397
Epoch 67/150
1051/1051 [==============================

Epoch 120/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0139 - mse: 0.0139 - val_loss: 0.1497 - val_mse: 0.1497
Epoch 121/150
1051/1051 [==============================] - 0s 45us/step - loss: 0.0137 - mse: 0.0137 - val_loss: 0.1487 - val_mse: 0.1487
Epoch 122/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0139 - mse: 0.0139 - val_loss: 0.1496 - val_mse: 0.1496
Epoch 123/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0134 - mse: 0.0134 - val_loss: 0.1498 - val_mse: 0.1498
Epoch 124/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0136 - mse: 0.0136 - val_loss: 0.1480 - val_mse: 0.1480
Epoch 125/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0134 - mse: 0.0134 - val_loss: 0.1547 - val_mse: 0.1547
Epoch 126/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0131 - mse: 0.0131 - val_loss: 0.1495 - val_mse: 0.1495
Epoch 127/150
1051/1051 [=========

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [11]:
# Evaluate the model on training data
#evaluate model
results_train = normalized_model.evaluate(X_train, y_train_scaled)
print('----------')
print(f'Training Loss: {results_train[0]:.3} \nTraining Accuracy: {results_train[1]:.3}')

1051/1051 [==============================] - 0s 24us/step
----------
Training Loss: 0.0102 
Training Accuracy: 0.0102


In [12]:
results_train

[0.010157228163049177, 0.010157227516174316]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [13]:
# Evaluate the model on validate data
results_train = normalized_model.evaluate(X_val, y_val_scaled)
print('----------')
print(f'Training Loss: {results_train[0]:.3} \nTraining Accuracy: {results_train[1]:.3}')

263/263 [==============================] - 0s 38us/step
----------
Training Loss: 0.155 
Training Accuracy: 0.155


Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [15]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)
# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

30909.291135191033

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [19]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, activation='relu', 
                          input_shape=(n_features,),
                          kernel_initializer='he_normal'))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 103us/step - loss: 0.5880 - mse: 0.5880 - val_loss: 0.2512 - val_mse: 0.2512
Epoch 2/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.2799 - mse: 0.2799 - val_loss: 0.1970 - val_mse: 0.1970
Epoch 3/150
1051/1051 [==============================] - 0s 59us/step - loss: 0.2158 - mse: 0.2158 - val_loss: 0.2011 - val_mse: 0.2011
Epoch 4/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.1875 - mse: 0.1875 - val_loss: 0.1556 - val_mse: 0.1556
Epoch 5/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.1613 - mse: 0.1613 - val_loss: 0.1509 - val_mse: 0.1509
Epoch 6/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.1408 - mse: 0.1408 - val_loss: 0.1404 - val_mse: 0.1404
Epoch 7/150
1051/1051 [==============================] - 0s 62us/step - loss: 0.1345 - mse: 0.1345 - val_loss: 0.1568 - val_mse: 0.1568


1051/1051 [==============================] - 0s 52us/step - loss: 0.0269 - mse: 0.0269 - val_loss: 0.1467 - val_mse: 0.1467
Epoch 61/150
1051/1051 [==============================] - 0s 59us/step - loss: 0.0264 - mse: 0.0264 - val_loss: 0.1482 - val_mse: 0.1482
Epoch 62/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.0263 - mse: 0.0263 - val_loss: 0.1494 - val_mse: 0.1494
Epoch 63/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.0260 - mse: 0.0260 - val_loss: 0.1497 - val_mse: 0.1497
Epoch 64/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.0256 - mse: 0.0256 - val_loss: 0.1477 - val_mse: 0.1477
Epoch 65/150
1051/1051 [==============================] - 0s 55us/step - loss: 0.0252 - mse: 0.0252 - val_loss: 0.1474 - val_mse: 0.1474
Epoch 66/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.0244 - mse: 0.0244 - val_loss: 0.1480 - val_mse: 0.1480
Epoch 67/150
1051/1051 [==============================

Epoch 120/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0126 - mse: 0.0126 - val_loss: 0.1490 - val_mse: 0.1490
Epoch 121/150
1051/1051 [==============================] - 0s 55us/step - loss: 0.0127 - mse: 0.0127 - val_loss: 0.1482 - val_mse: 0.1482
Epoch 122/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0125 - mse: 0.0125 - val_loss: 0.1484 - val_mse: 0.1484
Epoch 123/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0124 - mse: 0.0124 - val_loss: 0.1482 - val_mse: 0.1482
Epoch 124/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0121 - mse: 0.0121 - val_loss: 0.1490 - val_mse: 0.1490
Epoch 125/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0121 - mse: 0.0121 - val_loss: 0.1485 - val_mse: 0.1485
Epoch 126/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.0120 - mse: 0.0120 - val_loss: 0.1488 - val_mse: 0.1488
Epoch 127/150
1051/1051 [=========

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on training data
he_results_train = he_model.evaluate(X_train, y_train_scaled)
print(f'Training he_model{he_results_train[0]:.3} \nTraining Accuracy: {he_results_train[1]:.3}')

1051/1051 [==============================] - 0s 25us/step
Training he_model0.00883 
Training Accuracy: 0.00883


Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [24]:
X_test.shape

(263, 296)

In [25]:
y_test_scaled.shape

(146, 1)

In [28]:
# Evaluate the model on validate data
he_results_val = he_model.evaluate(X_val, y_val_scaled)
print(f'Val  he_model:  {he_results_val[0]:.3} \nVal he_model Accuracy: {he_results_val[1]:.3}')

263/263 [==============================] - 0s 36us/step
Val  he_model:  0.152 
Val he_model Accuracy: 0.152


## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [30]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, activation='relu', 
                            input_shape=(n_features,),
                            kernel_initializer='lecun_normal'))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 107us/step - loss: 0.5336 - mse: 0.5336 - val_loss: 0.1963 - val_mse: 0.1963
Epoch 2/150
1051/1051 [==============================] - 0s 54us/step - loss: 0.2333 - mse: 0.2333 - val_loss: 0.1489 - val_mse: 0.1489
Epoch 3/150
1051/1051 [==============================] - 0s 59us/step - loss: 0.1834 - mse: 0.1834 - val_loss: 0.1503 - val_mse: 0.1503
Epoch 4/150
1051/1051 [==============================] - 0s 66us/step - loss: 0.1572 - mse: 0.1572 - val_loss: 0.1155 - val_mse: 0.1155
Epoch 5/150
1051/1051 [==============================] - 0s 60us/step - loss: 0.1378 - mse: 0.1378 - val_loss: 0.1138 - val_mse: 0.1138
Epoch 6/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.1208 - mse: 0.1208 - val_loss: 0.1118 - val_mse: 0.1118
Epoch 7/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.1137 - mse: 0.1137 - val_loss: 0.1150 - val_mse: 0.1150


1051/1051 [==============================] - 0s 50us/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1070 - val_mse: 0.1070
Epoch 61/150
1051/1051 [==============================] - 0s 54us/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 62/150
1051/1051 [==============================] - 0s 53us/step - loss: 0.0234 - mse: 0.0234 - val_loss: 0.1060 - val_mse: 0.1060
Epoch 63/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.0232 - mse: 0.0232 - val_loss: 0.1063 - val_mse: 0.1063
Epoch 64/150
1051/1051 [==============================] - 0s 47us/step - loss: 0.0227 - mse: 0.0227 - val_loss: 0.1075 - val_mse: 0.1075
Epoch 65/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.0224 - mse: 0.0224 - val_loss: 0.1045 - val_mse: 0.1045
Epoch 66/150
1051/1051 [==============================] - 0s 55us/step - loss: 0.0216 - mse: 0.0216 - val_loss: 0.1098 - val_mse: 0.1098
Epoch 67/150
1051/1051 [==============================

Epoch 120/150
1051/1051 [==============================] - 0s 52us/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1064 - val_mse: 0.1064
Epoch 121/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1041 - val_mse: 0.1041
Epoch 122/150
1051/1051 [==============================] - 0s 51us/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 123/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0108 - mse: 0.0108 - val_loss: 0.1041 - val_mse: 0.1041
Epoch 124/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.0107 - mse: 0.0107 - val_loss: 0.1032 - val_mse: 0.1032
Epoch 125/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.0105 - mse: 0.0105 - val_loss: 0.1062 - val_mse: 0.1062
Epoch 126/150
1051/1051 [==============================] - 0s 46us/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 127/150
1051/1051 [=========

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [49]:
# Evaluate the model on training data
lecun_train_results = lecun_model.evaluate(X_train, y_train_scaled)
print(lecun_train_results)

1051/1051 [==============================] - 0s 28us/step
[0.007434781819323236, 0.0074347821064293385]


Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [55]:
# Evaluate the model on validate data
lecun_val_results = lecun_model.evaluate(X_val, y_val_scaled)
print(lecun_val_results)

263/263 [==============================] - 0s 36us/step
[0.10566887194326169, 0.10566887259483337]


Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [35]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop',
               loss='mse',
               metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 130us/step - loss: 0.4161 - mse: 0.4161 - val_loss: 0.1333 - val_mse: 0.1333
Epoch 2/150
1051/1051 [==============================] - 0s 57us/step - loss: 0.1901 - mse: 0.1901 - val_loss: 0.1068 - val_mse: 0.1068
Epoch 3/150
1051/1051 [==============================] - 0s 64us/step - loss: 0.1405 - mse: 0.1405 - val_loss: 0.1384 - val_mse: 0.1384
Epoch 4/150
1051/1051 [==============================] - 0s 66us/step - loss: 0.1100 - mse: 0.1100 - val_loss: 0.1193 - val_mse: 0.1193
Epoch 5/150
1051/1051 [==============================] - 0s 66us/step - loss: 0.0887 - mse: 0.0887 - val_loss: 0.1467 - val_mse: 0.1467
Epoch 6/150
1051/1051 [==============================] - 0s 66us/step - loss: 0.0753 - mse: 0.0753 - val_loss: 0.1160 - val_mse: 0.1160
Epoch 7/150
1051/1051 [==============================] - 0s 68us/step - loss: 0.0580 - mse: 0.0580 - val_loss: 0.1207 - val_mse: 0.1207


1051/1051 [==============================] - 0s 51us/step - loss: 0.0079 - mse: 0.0079 - val_loss: 0.1283 - val_mse: 0.1283
Epoch 61/150
1051/1051 [==============================] - 0s 55us/step - loss: 0.0083 - mse: 0.0083 - val_loss: 0.1326 - val_mse: 0.1326
Epoch 62/150
1051/1051 [==============================] - 0s 61us/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1159 - val_mse: 0.1159
Epoch 63/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1159 - val_mse: 0.1159
Epoch 64/150
1051/1051 [==============================] - ETA: 0s - loss: 0.0078 - mse: 0.007 - 0s 55us/step - loss: 0.0072 - mse: 0.0072 - val_loss: 0.1296 - val_mse: 0.1296
Epoch 65/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.0101 - mse: 0.0101 - val_loss: 0.1135 - val_mse: 0.1135
Epoch 66/150
1051/1051 [==============================] - 0s 55us/step - loss: 0.0067 - mse: 0.0067 - val_loss: 0.1245 - val_mse: 0.1245
Epoch 67/150
105

1051/1051 [==============================] - 0s 60us/step - loss: 0.0061 - mse: 0.0061 - val_loss: 0.0934 - val_mse: 0.0934
Epoch 120/150
1051/1051 [==============================] - 0s 54us/step - loss: 0.0063 - mse: 0.0063 - val_loss: 0.1015 - val_mse: 0.1015
Epoch 121/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.0046 - mse: 0.0046 - val_loss: 0.1135 - val_mse: 0.1135
Epoch 122/150
1051/1051 [==============================] - 0s 50us/step - loss: 0.0061 - mse: 0.0061 - val_loss: 0.0998 - val_mse: 0.0998
Epoch 123/150
1051/1051 [==============================] - 0s 49us/step - loss: 0.0053 - mse: 0.0053 - val_loss: 0.0985 - val_mse: 0.0985
Epoch 124/150
1051/1051 [==============================] - 0s 55us/step - loss: 0.0052 - mse: 0.0052 - val_loss: 0.1039 - val_mse: 0.1039
Epoch 125/150
1051/1051 [==============================] - 0s 48us/step - loss: 0.0053 - mse: 0.0053 - val_loss: 0.1026 - val_mse: 0.1026
Epoch 126/150
1051/1051 [=======================

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [46]:
# Evaluate the model on training data
rmsprop_train_results = rmsprop_model.evaluate(X_train, y_train_scaled)
print(rmsprop_train_results)

1051/1051 [==============================] - 0s 24us/step
[0.0030938751614970695, 0.003093875013291836]


Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [54]:
# Evaluate the model on validate data
rmsprop_val_results = rmsprop_model.evaluate(X_val, y_val_scaled)
print(rmsprop_val_results)

263/263 [==============================] - 0s 42us/step
[0.0961499644776261, 0.09614996612071991]


## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [40]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam',
                  loss='mse',
                  metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Train on 1051 samples, validate on 263 samples
Epoch 1/150
1051/1051 [==============================] - 0s 173us/step - loss: 0.4778 - mse: 0.4778 - val_loss: 0.1353 - val_mse: 0.1353
Epoch 2/150
1051/1051 [==============================] - 0s 65us/step - loss: 0.1787 - mse: 0.1787 - val_loss: 0.0876 - val_mse: 0.0876
Epoch 3/150
1051/1051 [==============================] - 0s 76us/step - loss: 0.1194 - mse: 0.1194 - val_loss: 0.0941 - val_mse: 0.0941
Epoch 4/150
1051/1051 [==============================] - 0s 71us/step - loss: 0.0893 - mse: 0.0893 - val_loss: 0.0965 - val_mse: 0.0965
Epoch 5/150
1051/1051 [==============================] - 0s 73us/step - loss: 0.0755 - mse: 0.0755 - val_loss: 0.1085 - val_mse: 0.1085
Epoch 6/150
1051/1051 [==============================] - 0s 79us/step - loss: 0.0598 - mse: 0.0598 - val_loss: 0.1103 - val_mse: 0.1103
Epoch 7/150
1051/1051 [==============================] - 0s 67us/step - loss: 0.0518 - mse: 0.0518 - val_loss: 0.1072 - val_mse: 0.1072


1051/1051 [==============================] - 0s 75us/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.1306 - val_mse: 0.1306
Epoch 61/150
1051/1051 [==============================] - 0s 67us/step - loss: 7.3952e-04 - mse: 7.3952e-04 - val_loss: 0.1307 - val_mse: 0.1307
Epoch 62/150
1051/1051 [==============================] - 0s 65us/step - loss: 5.8277e-04 - mse: 5.8277e-04 - val_loss: 0.1307 - val_mse: 0.1307
Epoch 63/150
1051/1051 [==============================] - 0s 60us/step - loss: 3.8155e-04 - mse: 3.8155e-04 - val_loss: 0.1313 - val_mse: 0.1313
Epoch 64/150
1051/1051 [==============================] - 0s 59us/step - loss: 3.8417e-04 - mse: 3.8417e-04 - val_loss: 0.1318 - val_mse: 0.1318
Epoch 65/150
1051/1051 [==============================] - 0s 69us/step - loss: 3.0538e-04 - mse: 3.0538e-04 - val_loss: 0.1309 - val_mse: 0.1309
Epoch 66/150
1051/1051 [==============================] - 0s 80us/step - loss: 2.1565e-04 - mse: 2.1565e-04 - val_loss: 0.1319 - val_mse: 0.1319
Epoch 

1051/1051 [==============================] - 0s 71us/step - loss: 2.7656e-04 - mse: 2.7656e-04 - val_loss: 0.1174 - val_mse: 0.1174
Epoch 119/150
1051/1051 [==============================] - 0s 73us/step - loss: 1.7213e-04 - mse: 1.7213e-04 - val_loss: 0.1183 - val_mse: 0.1183
Epoch 120/150
1051/1051 [==============================] - 0s 62us/step - loss: 1.5793e-04 - mse: 1.5793e-04 - val_loss: 0.1174 - val_mse: 0.1174
Epoch 121/150
1051/1051 [==============================] - 0s 57us/step - loss: 1.2742e-04 - mse: 1.2742e-04 - val_loss: 0.1183 - val_mse: 0.1183
Epoch 122/150
1051/1051 [==============================] - 0s 56us/step - loss: 1.0882e-04 - mse: 1.0882e-04 - val_loss: 0.1179 - val_mse: 0.1179
Epoch 123/150
1051/1051 [==============================] - 0s 60us/step - loss: 7.8195e-05 - mse: 7.8195e-05 - val_loss: 0.1189 - val_mse: 0.1189
Epoch 124/150
1051/1051 [==============================] - 0s 60us/step - loss: 7.3391e-05 - mse: 7.3391e-05 - val_loss: 0.1174 - val_mse:

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [52]:
# Evaluate the model on training data
adam_train_res = adam_model.evaluate(X_train, y_train_scaled)
print(adam_train_res)

1051/1051 [==============================] - 0s 22us/step
[0.012159104615832713, 0.012159104458987713]


Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [53]:
# Evaluate the model on validate data
adam_val_res = adam_model.evaluate(X_val, y_val_scaled)
print(adam_val_res)

263/263 [==============================] - 0s 40us/step
[0.11304149252606889, 0.11304149031639099]


## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [60]:
# Evaluate the best model on test datahe_results_train
print(f'he: {he_results_train} , {he_results_val}')
print('==================')
print(f'rms: {rmsprop_train_results} , {rmsprop_val_results}')
print('==================')
print(f'lecun: {lecun_train_results} , {lecun_val_results}')
print('==================')
print(f'adam: {adam_train_res} , {adam_val_res}')

he: [0.008825844264971173, 0.008825844153761864] , [0.15188419886868262, 0.15188419818878174]
rms: [0.0030938751614970695, 0.003093875013291836] , [0.0961499644776261, 0.09614996612071991]
lecun: [0.007434781819323236, 0.0074347821064293385] , [0.10566887194326169, 0.10566887259483337]
adam: [0.012159104615832713, 0.012159104458987713] , [0.11304149252606889, 0.11304149031639099]


In [66]:
X_test.shape

(263, 296)

In [64]:
y_test_scaled.shape

(146, 1)

In [61]:
adam_model.evaluate(X_test, y_test_scaled)

ValueError: Input arrays should have the same number of samples as target arrays. Found 263 input samples and 146 target samples.

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [75]:
# Generate predictions on test data
y_test_pred_scaled = adam_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
MSE = np.sqrt(mean_squared_error(y_test,y_test_pred))
print(MSE)

24462518.59931303


## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.